In [ ]:
from eth_rpc import *

# create a new wallet
wallet = PrivateKeyWallet.create_new()
# print your private key so you can write it down for use later
print(wallet.address, wallet.private_key)

In [ ]:
from emp.orderly.onboarding import create_account, add_access_key

# create a new account on orderly
await create_account(wallet)

In [13]:
from emp.orderly.utils.ed25519 import publickey
from base58 import b58encode

# NOTE: you can make as many orderly keys as you want, but this one is generated from your
#       wallets private key.  Keep this PRIVATE!
key_bytes = publickey(bytes.fromhex(wallet.private_key.removeprefix("0x")))
orderly_key = "ed25519:%s" % b58encode(key_bytes).decode("utf-8")

In [ ]:
# register your new orderly key as a signer for your wallet address
await add_access_key(wallet, orderly_key)

In [ ]:
from emp.orderly.onboarding.faucet import request_testnet_funds

# get some testnet USDC from the faucet
await request_testnet_funds(wallet)

In [16]:
from emp.orderly import EmpyrealOrderlySDK
from emp.orderly.utils.orderly_id import from_address

# now you can initialize the SDK
sdk = EmpyrealOrderlySDK(pvt_hex=wallet.private_key, account_id=from_address(wallet.address), is_testnet=True)

In [ ]:
from decimal import Decimal
from emp.orderly_types import PerpetualAssetType, MarketInfo, OrderResponse, PositionData
from pprint import pprint

bitcoin_asset_info = await sdk.asset_info(PerpetualAssetType.BTC)
pprint(bitcoin_asset_info)

In [ ]:
# this is the min tick size for an asset
base_min = Decimal(str(bitcoin_asset_info['data']['base_min']))
quote_tick = Decimal(str(bitcoin_asset_info['data']['quote_tick']))

print(f"base_min: {base_min}\nquote_tick: {quote_tick}")

In [ ]:
bitcoin_market_info: MarketInfo = await sdk.get_market_info(PerpetualAssetType.BTC)
print(bitcoin_market_info)

In [ ]:
# the market info has information about the asset, such as its index price, mark price, vunding rates, volume, and more
bitcoin_price = bitcoin_market_info.index_price
print(bitcoin_price)

In [ ]:
# we want to buy $25, so let's divide 25 by the bitcoin price.  We're using the decimal module to keep precision in our
# decimal numbers
purchase_amount = (Decimal('25') / bitcoin_price).quantize(base_min)
print(purchase_amount)

In [ ]:
# let's make the trade and open our position
response: OrderResponse = await sdk.make_order( purchase_amount, PerpetualAssetType.BTC, is_buy=True)
positions: PositionData = await sdk.positions()
print(response, positions)

In [ ]:
# now we can see our current bitcoin position (assuming we have no other positions open)
print([position for position in positions.rows if position.symbol == PerpetualAssetType.BTC])

In [ ]:
take_profit_price: Decimal = (bitcoin_price * Decimal('1.1')).quantize(quote_tick)
await sdk.make_limit_order('0.0005', price=take_profit_price, asset=PerpetualAssetType.BTC, is_buy=False)
print(take_profit_price)

In [ ]:
from emp.orderly_types import Order

# get all unfulfilled orders to see your open bitcoin order
orders: list[Order] = await sdk.orders(status="INCOMPLETE")
print(orders)

In [ ]:
# now, let's close our open limit order
await sdk.close_orders(PerpetualAssetType.BTC)

In [ ]:
# and we can close our bitcoin position
await sdk.close_position(PerpetualAssetType.BTC)